### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np
import datetime
import random
import json

In [3]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [7]:
scope = ["https://spreadsheets.google.com/feeds"]
# credentials = ServiceAccountCredentials.from_json_keyfile_name(
#    "jupyter-integration-credentials-redistricting.json", scope
# )
# gc = gspread.authorize(credentials)

source: https://twitter.com/ddiamond/status/1555969337068081154<br>
data series:<br>
'CES6562110001' (physicians offices)<br>
'CES6562140001' (outpatient care centers)<br>
'CES6562200001' (hospitals)<br>
'CES6562300001' (nursing homes)<br>

### read in bls data from FRED

Physicians offices

In [8]:
with urllib.request.urlopen(
    "https://api.stlouisfed.org/fred/series/observations?series_id=CES6562110001&api_key=b98b54e0ff7221aca6e1b161f5b12435&file_type=json"
) as url:
    data = json.loads(url.read().decode())

In [9]:
dataframes = []
for d in data["observations"]:
    dataframes.append(pd.DataFrame(d, index=[0]))

In [10]:
physicians_office = pd.concat(dataframes).reset_index(drop=True)

Outpatient care centers

In [11]:
with urllib.request.urlopen(
    "https://api.stlouisfed.org/fred/series/observations?series_id=CES6562140001&api_key=b98b54e0ff7221aca6e1b161f5b12435&file_type=json"
) as url:
    data = json.loads(url.read().decode())

In [12]:
dataframes = []
for d in data["observations"]:
    dataframes.append(pd.DataFrame(d, index=[0]))

In [13]:
outpatient = pd.concat(dataframes).reset_index(drop=True)

Hospitals

In [14]:
with urllib.request.urlopen(
    "https://api.stlouisfed.org/fred/series/observations?series_id=CES6562200001&api_key=b98b54e0ff7221aca6e1b161f5b12435&file_type=json"
) as url:
    data = json.loads(url.read().decode())

In [15]:
dataframes = []
for d in data["observations"]:
    dataframes.append(pd.DataFrame(d, index=[0]))

In [16]:
hospitals = pd.concat(dataframes).reset_index(drop=True)

Nursing homes

In [17]:
with urllib.request.urlopen(
    "https://api.stlouisfed.org/fred/series/observations?series_id=CES6562300001&api_key=b98b54e0ff7221aca6e1b161f5b12435&file_type=json"
) as url:
    data = json.loads(url.read().decode())

In [18]:
dataframes = []
for d in data["observations"]:
    dataframes.append(pd.DataFrame(d, index=[0]))

In [19]:
nursing_homes = pd.concat(dataframes).reset_index(drop=True)

### clean dfs

In [20]:
physicians_office = physicians_office.drop(columns=["realtime_start", "realtime_end"])
nursing_homes = nursing_homes.drop(columns=["realtime_start", "realtime_end"])
hospitals = hospitals.drop(columns=["realtime_start", "realtime_end"])
outpatient = outpatient.drop(columns=["realtime_start", "realtime_end"])

In [21]:
physicians_office["type"] = "Physicians' offices"
nursing_homes["type"] = "Nursing homes"
hospitals["type"] = "Hospitals"
outpatient["type"] = "Outpatient care"

#### combining each venue into single df and changing the types of variables

In [22]:
df_list = [physicians_office, nursing_homes, hospitals, outpatient]

In [23]:
combined = pd.concat(df_list)

In [24]:
combined["date"] = pd.to_datetime(combined["date"])
combined["value"] = combined["value"].astype(float)
combined["type"] = combined["type"].astype(str)

#### if you want to treat each venue separately

In [25]:
for df in df_list:
    df["date"] = pd.to_datetime(df["date"])
    df["value"] = df["value"].astype(float)
    df["type"] = df["type"].astype(str)

### charting

In [26]:
alt.Chart(combined[combined["date"] >= "2020-01-01"]).mark_line().encode(
    x=alt.X("date"), y=alt.Y("value"), color=alt.Color("type")
)

alt.Chart(...)

In [27]:
alt.Chart(combined[combined["date"] >= "2020-01-01"]).mark_line().encode(
    x=alt.X("date"), y=alt.Y("value")
).properties(width=400, height=200).facet(facet=alt.Facet("type:O"), columns=2)

alt.FacetChart(...)

### rolling avg 12 month pct change


In [28]:
for df in df_list:
    df.sort_values("date", ascending=True)
    df["pct_change"] = df["value"].pct_change()

In [29]:
for df in df_list:
    df.sort_values("date", ascending=True)
    df["rollmean"] = df["pct_change"].rolling(12).mean()

In [30]:
combined_rollmean = pd.concat(df_list)

In [31]:
alt.Chart(
    combined_rollmean[combined_rollmean["date"] >= "2020-01-01"]
).mark_bar().encode(x=alt.X("date"), y=alt.Y("rollmean")).properties(
    width=400, height=200
).facet(
    facet=alt.Facet("type:O"), columns=2
)

alt.FacetChart(...)